In [1]:
import cv2
import numpy as np
import time 
from PIL import Image, ImageFilter
1.
sr = cv2.imread('H:/edge_dection/Jockey_001/SR/001.png',0)#1彩图，0灰度图
imgH = cv2.imread('H:/edge_dection/Jockey_001/sharp/001.png',1)
hr = cv2.cvtColor (imgH,cv2.COLOR_BGR2GRAY)
'''
hr = Image.open('H:/edge_dection/Ready_008/sharp/001.png').convert('L')  # 打开图片文件
hr = hr.filter(ImageFilter.CONTOUR).convert('L')  # 设置图片轮廓筛选器 
hr.save('H:/edge_dection/Ready_008/轮廓效果h.png', 'png') 
sr = Image.open('H:/edge_dection/Ready_008/SR/001.png').convert('L')  # 打开图片文件
sr = sr.filter(ImageFilter.CONTOUR).convert('L')  # 设置图片轮廓筛选器 
sr.save('H:/edge_dection/Ready_008/轮廓效果s.png', 'png')
err = np.array(hr)-np.array(sr)
cv2.namedWindow('sr', cv2.WINDOW_NORMAL)
cv2.resizeWindow("sr", 800, 500) 
cv2.imshow("sr", err)
'''

#sr = cv2.GaussianBlur(sr,(5,5),0)
sr= cv2.Canny(sr,100,110)
#sr = cv2.adaptiveThreshold(sr,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,401,-10)#401,-10
contours, hierarchy = cv2.findContours(sr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
##cv2.namedWindow('sr', cv2.WINDOW_NORMAL)
##cv2.resizeWindow("sr", 800, 500) 
##cv2.imshow("sr", sr) 

#hr = cv2.GaussianBlur(hr,(5,5),0)
hr= cv2.Canny(hr,100,110)
#hr = cv2.adaptiveThreshold(hr,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,3)
contours, hierarchy = cv2.findContours(hr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
##cv2.namedWindow('hr', cv2.WINDOW_NORMAL)
##cv2.resizeWindow("hr", 800, 500) 
##cv2.imshow("hr", hr) 


#err = hr-sr
err = cv2.bitwise_xor(hr,sr)
cv2.namedWindow('err', cv2.WINDOW_NORMAL)
cv2.resizeWindow("err", 800, 500) 
cv2.imshow("err", err) 
print(err)

cv2.waitKey(0)
cv2.destroyAllWindows()


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [21]:
patch_size=600

def get_loc(img, patch_size):#########随机补丁位置
    ih, iw = img.shape[:2]
    #print(ih,iw)
    ip = patch_size
    C=[]
    R=[]
    L=[]
    for ix in range(0, iw - ip + 1,200):
        for iy in range(0,ih - ip + 1,200):
            #print(ix,iy)
            ret =np.array( 
                img[iy:iy + ip, ix:ix + ip]
            )   #隔200取一个，分块
            #print(ret.shape)
            #cv2.imwrite('H:/edge_dection/Ready_008/'+str(ix)+"_"+str(iy)+'.png',ret )
            
            contours, hierarchy = cv2.findContours(ret, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#查找检测物体的轮廓
            R.append([ix,iy])#保存每个块起始位置
            #print(len(contours))
            C.append(len(contours))#保存每个块轮廓数
            
    '''取最大轮廓数索引，找最大轮廓位置，保存前10个块'''
    print("总块数：",len(C))
    num=len(C)
    print("提取块数：",int(len(C)/2))
    for i in range(int(len(C))):###########10 patch        
        print("最大轮廓数：",max(C))####
        index=C.index(max(C))#####
        print("最大值索引：",index)
        print("位置：      ",R[index])
        L.append(R[index])
        del C[index]
        del R[index]
    return num,L

def get_patch(imgH,imgL, i,ix,iy,patch_size, scale=8):#########轮廓 Top max
    #start = time.time()
    ip = patch_size
    
    '''scale=8,取HR、LR对应位置块'''
    retH =np.array( 
        imgH[iy:iy + ip, ix:ix + ip,:]
    )   #hr patch
    print(iy/scale,ix/scale)
    print(iy,iy + ip, ix,ix + ip)
    retL =np.array( 
        imgL[int(iy/scale):int(iy/scale) + int(ip/scale), int(ix/scale):int(ix/scale) + int(ip/scale),:]
    )   #lr patch
    #end = time.time()
    #t=end-start
    #get_t.append(t)
    
    if i+1 <10:
        cv2.imwrite('H:/edge_dection/Jockey_001/sharp_patch_resd/00'+str(i+1)+'.png',retH )
        cv2.imwrite('H:/edge_dection/Jockey_001/LR_patch_resd/00'+str(i+1)+'.png',retL )
    else:
        cv2.imwrite('H:/edge_dection/Jockey_001/sharp_patch_resd/0'+str(i+1)+'.png',retH )
        cv2.imwrite('H:/edge_dection/Jockey_001/LR_patch_resd/0'+str(i+1)+'.png',retL )
   

#  源目录
'''
cv2.namedWindow('original', cv2.WINDOW_NORMAL)
cv2.resizeWindow("original", 800, 500) 
cv2.imshow("original", img) 
cv2.waitKey(0)
cv2.destroyAllWindows()
#print(img.size)
'''
    
start = time.time()
#2.
num,L= get_loc(err,patch_size)#10个块位置

#3.
#imgH = cv2.imread('H:/edge_dection/Ready_008/009.png')
imgL = cv2.imread('H:/edge_dection//Jockey_001/LR_bicubic/8x/001.png')
for i in range(num):###########10 patch ；i,j变
    ix = L[i][0]
    iy = L[i][1]
    get_patch(imgH,imgL,i,ix,iy,patch_size)
    
end = time.time()
print("循环运行时间:%.2f秒"%(end-start))
'''
Jockey_000 2.08秒 0.60秒 
Jockey_008 2.03秒 0.55秒
Ready_000  2.46秒 1.00秒
Ready_008  2.15秒 0.61秒
'''

总块数： 136
提取块数： 68
最大轮廓数： 3764
最大值索引： 36
位置：       [800, 800]
最大轮廓数： 3541
最大值索引： 96
位置：       [2400, 200]
最大轮廓数： 3477
最大值索引： 43
位置：       [1000, 800]
最大轮廓数： 3429
最大值索引： 28
位置：       [600, 800]
最大轮廓数： 3271
最大值索引： 34
位置：       [800, 600]
最大轮廓数： 3245
最大值索引： 27
位置：       [600, 600]
最大轮廓数： 3158
最大值索引： 39
位置：       [1000, 600]
最大轮廓数： 3152
最大值索引： 83
位置：       [2200, 200]
最大轮廓数： 3140
最大值索引： 97
位置：       [2600, 200]
最大轮廓数： 3018
最大值索引： 33
位置：       [800, 1000]
最大轮廓数： 2852
最大值索引： 38
位置：       [1000, 1000]
最大轮廓数： 2832
最大值索引： 44
位置：       [1200, 800]
最大轮廓数： 2831
最大值索引： 87
位置：       [2400, 400]
最大轮廓数： 2802
最大值索引： 43
位置：       [1200, 600]
最大轮廓数： 2733
最大值索引： 19
位置：       [400, 600]
最大轮廓数： 2732
最大值索引： 25
位置：       [600, 400]
最大轮廓数： 2713
最大值索引： 90
位置：       [2600, 400]
最大轮廓数： 2679
最大值索引： 25
位置：       [600, 1000]
最大轮廓数： 2649
最大值索引： 76
位置：       [2200, 400]
最大轮廓数： 2644
最大值索引： 46
位置：       [1400, 600]
最大轮廓数： 2601
最大值索引： 19
位置：       [400, 800]
最大轮廓数： 2543
最大值索引： 18
位置：       [400, 400]
最大轮廓数： 2536
最大值索引： 92

100.0 325.0
800 1400 2600 3200
125.0 275.0
1000 1600 2200 2800
100.0 350.0
800 1400 2800 3400
0.0 75.0
0 600 600 1200
100.0 375.0
800 1400 3000 3600
150.0 300.0
1200 1800 2400 3000
100.0 0.0
800 1400 0 600
0.0 350.0
0 600 2800 3400
150.0 175.0
1200 1800 1400 2000
125.0 25.0
1000 1600 200 800
150.0 275.0
1200 1800 2200 2800
125.0 325.0
1000 1600 2600 3200
125.0 250.0
1000 1600 2000 2600
125.0 200.0
1000 1600 1600 2200
0.0 100.0
0 600 800 1400
0.0 175.0
0 600 1400 2000
150.0 250.0
1200 1800 2000 2600
150.0 25.0
1200 1800 200 800
0.0 150.0
0 600 1200 1800
125.0 225.0
1000 1600 1800 2400
0.0 400.0
0 600 3200 3800
0.0 125.0
0 600 1000 1600
125.0 350.0
1000 1600 2800 3400
150.0 325.0
1200 1800 2600 3200
150.0 200.0
1200 1800 1600 2200
125.0 400.0
1000 1600 3200 3800
0.0 200.0
0 600 1600 2200
125.0 375.0
1000 1600 3000 3600
150.0 225.0
1200 1800 1800 2400
125.0 0.0
1000 1600 0 600
0.0 225.0
0 600 1800 2400
0.0 375.0
0 600 3000 3600
150.0 0.0
1200 1800 0 600
175.0 300.0
1400 2000 2400 3000
150

'\nJockey_000 2.08秒 0.60秒 \nJockey_008 2.03秒 0.55秒\nReady_000  2.46秒 1.00秒\nReady_008  2.15秒 0.61秒\n'

In [ ]:
#输入训练看效果